In [1]:
import math
import pandas as pd
from dataclasses import dataclass
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import numpy as np
from IPython.display import HTML, display
import tabulate

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

import nbimporter
from helpers import estimateCaliTax, estimateUSTax, calculateFica, estimateCompleteCaliTax


cf.go_offline()
init_notebook_mode(connected=True)

# def hide_code_option():
#     return HTML('''<script>
#         code_show=true; 
#         function code_toggle() {
#          if (code_show){
#          $('div.input').hide();
#          } else {
#          $('div.input').show();
#          }
#          code_show = !code_show
#         } 
#         $( document ).ready(code_toggle);
#         </script>
#         <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# hide_code_option()

Importing Jupyter notebook from helpers.ipynb


In [38]:
@dataclass
class plan:
    
    retirement_contribution_max = 19500
#   START Parameters that can be pass in
    salary: float
    retirement_contribution: float
    retirement_match: float
    espp_contribution: float
    budget: dict
    retirement_match_max: float = math.inf
    tax_rate: float = 35
    espp_discount: float=15
    average_annual_return: float = 7.96 # 5-10% is also okay
    retirement_average_annual_return: float = 5 # after taxes and inflation
    current_age: int = 23
    retirement_age: int = 50
    retirement_end: int = 100
    additional_monthly_savings: float = 1000
    retirement_monthly_spend: float  = 6000
    current_year_income: float = 0
#     END parameters

    def plan(self, months, simulation_years = None, start_date = None):
        if not simulation_years:
            simulation_years = self.retirement_age - self.current_age
            
        self.months = months

        self.income = self.salary / 12 * months
        self.year_1_income = self.income + self.current_year_income
        self.self_retirement_contribution = min(self.retirement_contribution_max, self.income * (self.retirement_contribution / 100.0))
        self.employor_retirement_contribution = min(self.income * (self.retirement_match / 100.0), self.retirement_match_max)
        self.retirement_savings =  self.self_retirement_contribution + self.employor_retirement_contribution
        self.taxable_income = self.year_1_income - self.self_retirement_contribution
        
        self.real_tax_estimate = (estimateCaliTax(self.taxable_income) + estimateUSTax(self.taxable_income) + calculateFica(self.income + self.current_year_income))
        self.retirement_tax_savings = estimateCompleteCaliTax(self.year_1_income) - self.real_tax_estimate

        self.taxes = (self.taxable_income) * (self.tax_rate / 100.0)
        self.posttax_income = self.taxable_income - self.taxes
        self.espp = self.posttax_income * (self.espp_contribution / 100.0)
        
        self.pay_cheque = self.posttax_income - self.espp
        
        self.monthly_savings = self.pay_cheque / self.months
        
        for k,v in self.budget.items():
            self.monthly_savings -= v
        
        if start_date == None:
            start_day = datetime.today().replace(minute=0, hour=0, second=0, microsecond=0, day=1, tzinfo=None)
            simulate_months = simulation_years * 12 + (12 - start_day.month)
        
        
            
        df = pd.DataFrame()
        df["date"] = pd.date_range(start=start_day, periods=simulate_months, freq="M", tz=None)
        
        df["Employee contribution"] = self.salary / 12 * (self.retirement_contribution / 100.0)
        df["Employor contribution"] = self.salary / 12 * (self.retirement_match / 100.0)
        
        df["Contribution"] = df["Employee contribution"][0] + df["Employor contribution"][0]
                
        employee_yearly_sum = 0
        employor_yearly_sum = 0
        for i in range(1, len(df)):
            if df.loc[i, "date"].month == 1:
                employee_yearly_sum = 0
                employor_yearly_sum = 0
            employee_yearly_sum += df.at[i, "Employee contribution"]
            employor_yearly_sum += df.at[i, "Employor contribution"]
            
            if employee_yearly_sum > self.retirement_contribution_max:
                if employee_yearly_sum - df.at[i, "Employee contribution"] < self.retirement_contribution_max:
                    df.loc[i, "Employee contribution"] = self.retirement_contribution_max - (employee_yearly_sum - df.at[i, "Employee contribution"]) 
                else:
                    df.loc[i, "Employee contribution"] = 0
                df.loc[i, "Employor contribution"] = 0
            if employor_yearly_sum > self.retirement_match_max:
                if employor_yearly_sum - df.at[i, "Employor contribution"] < self.retirement_match_max:
                    df.loc[i, "Employor contribution"] = self.retirement_match_max - (employor_yearly_sum - df.at[i, "Employor contribution"]) 
                else:
                    df.loc[i, "Employor contribution"] = 0

        df["Contribution"] = df["Employee contribution"] + df["Employor contribution"]
                    
        pv = np.ndarray(shape=(len(df.index),1), dtype=np.float)
        pv[0] = df["Contribution"].values[0]
        for i in range(1, len(df.index)):
            pv[i] = pv[i-1] * (1 + (self.average_annual_return/100/12)) + df.at[i, "Contribution"]

        df["Present value"] = pv

        df["Principal"] = df["Contribution"].cumsum()

        self.retirement_df = df
        
        self.total_saved = self.espp + self.retirement_savings + self.monthly_savings * self.months
        
        self.monthly_summary = {
            "Income": self.income / self.months,
            "Employee 401K Contribution": self.self_retirement_contribution / self.months,
            "Employor 401K Contribution": self.employor_retirement_contribution / self.months,
            "401K Contribution": self.retirement_savings / self.months,
            "Taxes": self.taxes / self.months,
            "ESPP": self.espp / self.months,
            **self.budget,
            "Savings": self.monthly_savings,
            "Pay cheque": self.pay_cheque / self.months,
            "Budget Total": sum(self.budget.values()) / 2,
            "Total saved (401k+espp+savings)": (self.total_saved) / self.months / 2
        }
        
        
        return self
    
    def planRetirement(self):
        if self.additional_monthly_savings > self.monthly_savings:
            print("Warning: additional monthly savings is more than your left over monthly savings")
            
        df = self.retirement_df.copy()
        
        df["Additional Contribution"] = self.additional_monthly_savings
        df["Contribution"] = df["Employee contribution"] + df["Employor contribution"] + df["Additional Contribution"]
                            
        pv = np.ndarray(shape=(len(df.index),1), dtype=np.float)
        pv[0] = df["Contribution"].values[0]
        for i in range(1, len(df.index)):
            pv[i] = pv[i-1] * (1 + (self.average_annual_return/100/12)) + df.at[i, "Contribution"]

        df["Present value"] = pv

        df["Principal"] = df["Contribution"].cumsum()
        
        r_df = pd.DataFrame()
        r_df["date"] = pd.date_range(start=df.iloc[-1]["date"], periods=(self.retirement_end - self.retirement_age) * 12, freq="M", tz=None)
        r_df["Contribution"] = -1 * self.retirement_monthly_spend
        
        pv = np.ndarray(shape=(len(r_df.index),1), dtype=np.float)
        pv[0] = df["Present value"].values[-1]
        for i in range(1, len(r_df.index)):
            pv[i] = pv[i-1] * (1 + (self.retirement_average_annual_return/100/12)) + r_df.at[i, "Contribution"]
        
        r_df["Present value"] = pv

        r_df.drop(df.index[0])
        
        self.retirement_plan = pd.concat([df,r_df],ignore_index=True, sort=False)
        
        return self
        
    
    def graphRetirementPlan(self):
        self.retirement_plan.iplot(x="date", title="Retirement Plan")
        return self
    
    def graphBasics(self):
        labels = ['Taxes','401K contribution','401K employer contribution','ESPP', 'Pay cheque']
        values = [self.taxes, 
                  self.self_retirement_contribution, 
                  self.employor_retirement_contribution, 
                  self.espp, 
                  self.pay_cheque]

        fig = go.Figure(data=[go.Pie(labels=labels, values=values, sort=False)])
        fig.show()
        
        return self   
    
    def printBasics(self):
        basics = {
            "Income": self.income,
            "Current Year Income": self.year_1_income,
            "Estimated taxes": self.taxes,
            "Real tax estimate": self.real_tax_estimate,
            "Estimated tax refund": self.taxes - self.real_tax_estimate,
            "Real estimated average tax rate": self.printPercent(self.real_tax_estimate / self.taxable_income),
            "": "",
            "401k employee contributions": self.self_retirement_contribution,
            "401k employor contributions": self.employor_retirement_contribution,
            "401k contributions": self.retirement_savings,
            "401k employee contribution rate": self.printPercent(self.self_retirement_contribution / self.income),
            "401k contribution rate": self.printPercent(self.retirement_savings / self.income),
            "401k contributions tax savings": self.retirement_tax_savings,
            " ": "",
            "ESPP contributions": self.espp,
            "ESPP employor contribution": self.espp * self.espp_discount / 100,
            "  ": "",
            "Pay cheque total": self.pay_cheque,
            "Bi-monthly cheque total": self.pay_cheque / self.months/2,
            "Savings": self.monthly_savings * self.months
        }
        
        if self.current_year_income == 0:
            del basics["Current Year Income"]
            
        self.printDictAsTable(basics)
        if (self.income * (self.retirement_contribution / 100.0) > self.retirement_contribution_max):
            print("Warning: Contributing more than allowed to 401k")
        
        if (self.monthly_savings < 0):
            print("ERROR: Spending more than is being earned")
        
        return self
    
    def graphBudget(self):
        labels = list(map(str, self.budget.keys()))
        values = list(map(float, self.budget.values()))

        fig = go.Figure(data=[go.Pie(labels=labels, values=values, sort=False)])
        fig.show()
        
        return self
    
    def printBudget(self):
        self.printDictAsTable(self.budget)
        
        return self
    
    def graphDetailed(self):
        labels = ['Taxes','401K contribution','401K employer contribution','ESPP']
        values = [self.taxes, 
                  self.self_retirement_contribution, 
                  self.employor_retirement_contribution, 
                  self.espp]
        
        
        for k,v in self.budget.items():
            labels.append(k)
            values.append(v * self.months)
        
        labels.append("Savings")
        values.append(self.monthly_savings * self.months)
            

        fig = go.Figure(data=[go.Pie(labels=labels, values=values, sort=False)])
        fig.show()
        
        return self
    
    def printDetailed(self):
        details = {
            "Income": self.income,
            "Current Year Income": self.year_1_income,
            "Estimated taxes": self.taxes,
            "401k contributions": self.retirement_savings,
            "ESPP contributions": self.espp,
            **dict((k, v * self.months) for k, v in self.budget.items()),
            "Savings": self.monthly_savings * self.months,
            "Total saved (401k+espp+savings)": self.total_saved
        }
        if self.current_year_income == 0:
            del details["Current Year Income"]
        
        self.printDictAsTable(details)
        return self
    
    monthlyBlackList = ["Income", "Pay cheque", "Total saved (401k+espp+savings)", "Budget Total"]
    def graphPaycheque(self):
        s = dict((k, v / 2.0) for k, v in self.monthly_summary.items())
        for e in self.monthlyBlackList:
            if e in s:
                del s[e]
        labels = list(map(str, s.keys()))
        values = list(map(float, s.values()))

        fig = go.Figure(data=[go.Pie(labels=labels, values=values, sort=False)])
        fig.show()
        
        return self
    
    def printPaycheque(self):
        s = dict((k, v / 2.0) for k, v in self.monthly_summary.items())
        self.printDictAsTable(s)
        
        return self
    

    def graphMonthly(self):
        monthly = self.monthly_summary.copy()
        for e in self.monthlyBlackList:
            if e in monthly:
                del monthly[e]
        labels = list(map(str, monthly.keys()))
        values = list(map(float, monthly.values()))

        fig = go.Figure(data=[go.Pie(labels=labels, values=values, sort=False)])
        fig.show()
        
        return self
    
    def printMonthly(self):
        self.printDictAsTable(self.monthly_summary)
        
        return self
    
    def graphRetirement(self):
        self.retirement_df.iplot(x="date", title="401k growth")
        return self
    
    def printRetirement(self):
        last_row = self.retirement_df.iloc[-1]
        year = self.retirement_df.at[1, "date"].year + 1
        year_sums = self.retirement_df[self.retirement_df.date.dt.year == year].sum()
        retire = {
            "Employee yearly contributions": year_sums["Employee contribution"],
            "Employor yearly contributions": year_sums["Employor contribution"],
            "Yearly contributions": year_sums["Contribution"],
            "Principal ({})".format(last_row["date"].year): last_row["Principal"],
            "Assumed annual rate of return": self.printPercent(self.average_annual_return / 100),
            "Present Value ({})".format(last_row["date"].year): last_row["Present value"],
        }
        self.printDictAsTable(retire)
        return self
    
    def graphRetirementTaxSavings(self):
        p = pd.DataFrame()
        p["401k Contribution"] = pd.Series(range(0, self.retirement_contribution_max + 1, 100))
        income = self.income + self.current_year_income
        base_tax = estimateCompleteCaliTax(income)
        p["savings"] = p["401k Contribution"].apply(lambda x: base_tax - ((estimateCaliTax(income - x) + estimateUSTax(income - x) + calculateFica(income))))
        # p["savings"] = p["income"].apply(lambda x: estimateCompleteCaliTax(x)-estimateCompleteCaliTax(x-19500))
        p.iplot(x="401k Contribution", title="Income vs Tax savings")
    
    def printDictAsTable(self, dic):
        arr = []
        
        for k,v in dic.items():
            if not isinstance(v, str):
                v = self.printDollars(v)
            arr.append([k, v])
        display(HTML("<style>table {margin-left: auto !important;margin-right: auto !important;}</style>" + tabulate.tabulate(arr, tablefmt='html')))
    
    def printPercent(self, p):
        return str(round(p * 10000)/100) + "%"
    
    def printDollars(self, p):
        return "$" + ('{:20,.2f}'.format(p)).strip()
    
    def printTitle(self, t):
        display(HTML("<h2>{}</h2>".format(t)))
        return(self)

In [ ]:
#   START Parameters that can be pass in
#     salary: float
#     retirement_contribution: float
#     retirement_match: float
#     espp_contribution: float
#     budget: dict
#     retirement_match_max: float = math.inf
#     tax_rate: float = 35
#     espp_discount: float=15
#     average_annual_return: float = 7.96 # 5-10% is also okay
#     retirement_average_annual_return: float = 5 # after taxes and inflation
#     current_age: int = 23
#     retirement_age: int = 50
#     retirement_end: int = 100
#     additional_monthly_savings: float = 1000
#     retirement_monthly_spend: float  = 6000
#     current_year_income: float = 0
#     END parameters

p = plan(salary=100000,
     retirement_contribution=10,
     retirement_match=6,
     retirement_match_max=50000,
     espp_contribution=5,
     budget = {
         "Housing": 2000,
         "Other": 2000
     },
     additional_monthly_savings = 0,
     retirement_monthly_spend = 5000,
     current_year_income = 0
    ).plan(months=4).planRetirement();

p.printTitle("Basics").graphBasics().printBasics();
p.printTitle("Budget").graphBudget().printBudget();
p.printTitle("Detailed").graphDetailed().printDetailed();
p.printTitle("Monthly").graphMonthly().printMonthly();
# p.printTitle("Pay cheque").graphPaycheque().printPaycheque();
p.printTitle("401K").graphRetirement().printRetirement();
p.graphRetirementTaxSavings();

p.printTitle("Retirement Plan").graphRetirementPlan();


Income,"$33,333.33"
Estimated taxes,"$10,500.00"
Real tax estimate,"$4,971.83"
Estimated tax refund,"$5,528.17"
Real estimated average tax rate,16.57%
,
401k employee contributions,"$3,333.33"
401k employor contributions,"$2,000.00"
401k contributions,"$5,333.33"
401k employee contribution rate,10.0%
401k contribution rate,16.0%


Housing,"$2,000.00"
Other,"$2,000.00"


Income,"$33,333.33"
Estimated taxes,"$10,500.00"
401k contributions,"$5,333.33"
ESPP contributions,$975.00
Housing,"$8,000.00"
Other,"$8,000.00"
Savings,"$2,525.00"
Total saved (401k+espp+savings),"$8,833.33"


Income,"$8,333.33"
Employee 401K Contribution,$833.33
Employor 401K Contribution,$500.00
401K Contribution,"$1,333.33"
Taxes,"$2,625.00"
ESPP,$243.75
Housing,"$2,000.00"
Other,"$2,000.00"
Savings,$631.25
Pay cheque,"$4,631.25"
Budget Total,"$2,000.00"


Employee yearly contributions,"$10,000.00"
Employor yearly contributions,"$6,000.00"
Yearly contributions,"$16,000.00"
Principal (2046),"$436,000.00"
Assumed annual rate of return,7.96%
Present Value (2046),"$1,545,317.40"


In [5]:
p = pd.DataFrame()
p["income"] = pd.Series(range(30000,500000, 1000))
p["tax"] = p["income"].apply(lambda x: estimateCompleteCaliTax(x))
p["tax with 401k"] = p["income"].apply(lambda x: (estimateCaliTax(x - 19500) + estimateUSTax(x - 19500) + calculateFica(x)))
p["savings"] = (p["tax"] - p["tax with 401k"])
# p["savings"] = p["income"].apply(lambda x: estimateCompleteCaliTax(x)-estimateCompleteCaliTax(x-19500))
p.iplot(x="income", title="Income vs Tax savings", columns=["savings"])
